# Introduction to Decision Trees

## Introduction

Decision trees are a powerful and popular machine learning technique. The basic concept is very similar to trees you may have seen commonly used to aid decision-making. Here's an example:

![](img/1.png)

In the diagram above, we're deciding whether we should wrestle a bear that's in front of us. We're using various criteria to make our final decision, including the size of the bear, and whether escape is possible. Let's say we had a data set of people who survived bear encounters and the actions they took:

```python
Bear name    Size    Escape possible?    Action
Yogi         Small   No                  Wrestle
Winnie       Small   Yes                 Wrestle
Baloo        Large   Yes                 Run away
Gentle Ben   Large   No                  Wrestle
```

If we wanted to optimize our chances of surviving a bear encounter, we could construct a decision tree to tell us what action to take.<br>

As our data set gets larger though, this becomes less and less practical. What if we had 10000 rows and 10 variables? Would you want to look through all of the possibilities to construct a tree?<br>

This is where the decision tree machine learning algorithm can help. It enables us to automatically construct a decision tree that tells us what outcomes we should predict in certain situations.<br>

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't. In our bear wrestling example, a decision tree can pick up on the fact that you should only wrestle large bears when escape is impossible, whereas a linear regression would have had to weight both factors in the absence of the other.<br>

We can use trees for classification or regression problems. In this lesson, we'll walk through the building blocks of making a decision tree automatically.


## Overview of the Dataset

We'll be looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year.

* You can download the data from [the University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult).

In [1]:
import pandas as pd

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pd.read_csv("data/income.csv", index_col=False)
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Converting Categorical Variables

As we can see in our data, we have categorical variables such as `workclass` that have string values. Multiple individuals can share the same string value. The types of work include `State-gov`, `Self-emp-not-inc`, `Private`, and so on. Each of these strings is a label for a category. Another example of a column of categories is `sex`, where the options are `Male` and `Female`.<br>

Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables. This involves assigning a number to each category label, then converting all of the labels in a column to the corresponding numbers.<br>

One strategy is to convert the columns to a [categorical](https://pandas-docs.github.io/pandas-docs-travis/categorical.html) type. Under this approach, pandas will display the labels as strings, but internally store them as numbers so we can do computations with them. The numbers aren't always compatible with other libraries like Scikit-learn, though, so it's easier to just do the conversion to numeric upfront.<br>

We can use the [Categorical.from_array](http://pandas.pydata.org/pandas-docs/version/0.15.2/generated/pandas.core.categorical.Categorical.from_array.html) method from pandas to perform the conversion to numbers.

In [2]:
cat_cols = [col for col in income.columns if income[col].dtype=='object']
cat_cols

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'high_income']

In [3]:
for cat in cat_cols:
    
    converted = pd.Categorical.from_array(income[cat])
    income[cat] = converted.codes

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


## Splitting Data

A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. Here's an example:

![](img/2.png)

In the diagram above, the node splits the data into two branches, `No` and `Yes`, based on whether the individual works in the private sector (the `workclass` column). We've mapped the value `Private` in the `workclass` column to the numeric code `4` (we can check this by comparing the values in the `workclass` column that used to have the label `Private` with the current values to see where they line up). So the `No` branch corresponds to `workclass != 4`, and the `Yes` branch corresponds to `workclass == 4`.<br>

This is exactly how a decision tree works -- we keep splitting the data based on variables. As we do this, the tree accumulates more levels. The tree we made above is two levels deep because it has one split, and two "levels" of nodes.<br>

The tree below is three levels deep.

![](img/3.png)

We added another split point to this tree "below", or after, our first split point. This made the tree three levels deep.


## Creating Splits

Now think of rows of data flowing through a decision tree. In the diagram above, we can split the data set into two portions based on whether the individual works in the private sector or not.

* Split `income` into two parts based on the value of the `workclass` column.
  * `private_incomes` should contain all rows where `workclass` is `4`.
  * `public_incomes` should contain all rows where `workclass` is not `4`.

In [5]:
private_incomes = income[income['workclass']==4]
public_incomes = income[income['workclass']!=4]

## Decision Trees as Flows of Data

When we performed the split, `9865` rows went to the left, where `workclass` does not equal `4`, and `22696` rows went to the right, where workclass equals `4`.<br>

It's useful to think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some will go to the left. As we build the tree deeper and deeper, each node will "receive" fewer and fewer rows.<br>

Here's a look at the splits, and the number of rows that will exist at each node:

![](img/4.png)

## Splitting Data to Make Predictions

The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves. When we do our splits, we aren't doing them randomly; we have an objective. Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.<br>

We're trying to predict the `high_income` column.
* If `high_income` is `1`, it means that the person has an income higher than 50k a year.
* If `high_income` is `0`, it means that they have an income less than or equal to 50k a year.

After constructing a tree using the `income` data, we'll want to make predictions. In order to do this, we'll take a new row and feed it through our decision tree.<br>

First, we check whether the person works in the private sector. If they do, we'll check to see whether they're native to the US, and so on.<br>

Eventually, we'll reach a leaf. The leaf will tell us what value we should predict for `high_income`.<br>

In order to be able to make this prediction, all of the rows in a leaf should only have a single value for `high_income`. This means that leaves can't have both `0` and `1` values in the high_income column. Each leaf can only have rows with the same values for our target column. If this isn't the case, we won't be able to make effective predictions.<br>

We'll need to continue splitting nodes until we get to a point where all of the rows in a node have the same value for `high_income`.

## Overview of Data Set Entropy

Now that we have a high-level view of how decision trees work, let's explore the details and learn how to perform the splits.<br>

We'll use a specific measure to figure out which variables we should split nodes on. Post-split, we'll have two data sets, each containing the rows from one branch of the split.<br>

Because we're trying to reach the leaves having only `1`s or only `0`s in `high_income`, each split will need to get us closer to that goal.<br>

When we split, we'll try to separate as many `0`s from `1`s in the `high_income` column as we can. In order to do this, we need a metric for how "together" the different values in the `high_income` column are.<br>

Data scientists commonly use a metric called **entropy** for this purpose. Entropy refers to **disorder**.
* The more "mixed together" `1`s and `0`s are, the higher the entropy.
* A data set consisting entirely of `1`s in the high_income column would have low entropy.

Entropy, which is not to be confused with entropy from physics, comes from [information theory](https://en.wikipedia.org/wiki/Information_theory). 
* Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. 
* A key concept in information theory is the notion of **a bit of information**. One bit of information is one [unit of information](https://en.wikipedia.org/wiki/Units_of_information).<br>

We can represent **a bit of information** as a **binary number** because it either has the value `1` or `0`. Suppose there's an equal probability of tomorrow being sunny (`1`) or not sunny (`0`). If I tell you that it will be sunny, I've given you one bit of information.<br>

We can also think of entropy in terms of information. 
* If we flip a coin where both sides are heads, 
  * we know upfront that the result will be heads. **We gain no new information by flipping the coin, so entropy is 0**. 
* On the other hand, if the coin has a heads side and a tails side,
  * there's a 50% probability that it will land on either. 
  * Thus, flipping the coin gives us one bit of information -- which side the coin landed on.

Entropy can be much more complex, especially when we get to cases with more than two possible outcomes, or differential probabilities. A deep understanding of entropy isn't necessary for constructing decision trees, however. If you'd like, you can read more about entropy [at Wikipedia](https://en.wikipedia.org/wiki/Entropy).<br>

The formula for entropy looks like this:

$$-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}$$

We iterate through each unique value in a single column (in this case, `high_income`), and assign it to i. We then compute the probability of that value occurring in the data ($P(x_i)$). Next we do some multiplication, and sum all of the values together. b is the base of the logarithm. We commonly use the value `2` for this, but we can also set it to `10` or another value.<br>

Let's say we have this data:

```python
age    high_income
25     1
50     1
30     0
50     0
80     1
```

We could compute its entropy like this:

$$
\begin{align}
-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)} \nonumber \\
-((2/5 * \log_2 2/5) + (3/5 * \log_2 3/5)) \nonumber \\ 
-(-0.5287712379549449 + -0.44217935649972373)\nonumber \\
.97 \nonumber \\
\end{align}
$$

We get less than one "bit" of information -- only `.97` -- because there are slightly more `1`s in the sample data than `0`s. This means that if we were predicting a new value, we could guess that the answer is `1` and be right more often than wrong (because there's a `.6` probability of the answer being `1`). 
* **Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value**.

## Overview of Data Set Entropy

* Compute the entropy of the `high_income` column in the `income` dataframe, and assign the result to `income_entropy`.


In [6]:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

0.9709505944546686


In [7]:
def calc_entropy(df, col):
    
    tot = df[col].value_counts().sum()
    ratios = [count/tot for count in df[col].value_counts()]
    entropy = 0
    
    for ratio in ratios:
        ent = ratio * math.log(ratio, 2)
        entropy += ent
        
    return -entropy

In [8]:
income['high_income'].value_counts()

0    24720
1     7841
Name: high_income, dtype: int64

In [9]:
calc_entropy(income, 'high_income')

0.79638395520221317

## Information Gain

We'll need a way to go from computing entropy to figuring out which variable to split on. We can do this using information gain, which tells us which split will reduce entropy the most.<br>

Here's the formula for information gain:

$$IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v})$$

It may look complicated, but we'll break it down. We're computing **information gain** ($IG$) for a given **target** variable ($T$), as well as a given **variable we want to split on** ($A$).<br>
1. To compute it, we first calculate the entropy for $T$. 
2. Then, for each unique value $v$ in the variable $A$, we compute the number of rows in which $A$ takes on the value $v$, and divide it by the total number of rows. 
3. Next, we multiply the results by the entropy of the rows where $A$ is $v$. 
4. We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

Here's an alternate explanation. We're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.<br>

One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.<br>

To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.<br>

Here's an example that uses the same data set we worked with earlier:

```python
age    high_income
25     1
50     1
30     0
50     0
80     1
```

Let's say we wanted to split this data set based on age. First, we calculate the median age, which is `50`. Then, we assign any row with a value less than or equal to the median age the value `0` (in a new column named `split_age`), and the other rows `1`.

```python
age    high_income    split_age
25     1              0
50     1              0
30     0              0
50     0              0
80     1              1
```

Now we compute entropy:


$$
\begin{align}
IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v}) \nonumber \\
.97 - (((4/5) * -(1/2 * log_{2} 1/2 + 1/2 * log_{2} 1/2 )) + -(1/5 *  (0 * log_{2} 0 + 1 * log_{2} 1 ))) \nonumber \\
 .97 - ((4/5) * -(-.5 + -.5)) + (1/5 *  -(0 + 1 * 0 )) \nonumber \\
.97 - (4/5) \nonumber \\
.169 \nonumber \\
\end{align}
$$

We end up with `.17` which means that we gain `.17` bits of information by splitting our data set on the `age` variable.


## Information Gain

Compute the information gain for splitting on the `age` column of `income`.
* First, compute the median of `age`.
* Then, assign anything less than or equal to the median to the left branch, and anything greater than the median to the right branch.
* Compute the information gain and assign it to `age_information_gain`.


* Re-define `calc_entropy` function
  * from : `takes in dataframe and column name`
  * to : `takes in column object itself`

In [31]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [32]:
import numpy as np

age_median = np.median(income['age'])
age_median

37.0

In [42]:
income_entropy = calc_entropy(income['high_income'])

age_left_split = income[income['age'] <= age_median]
age_right_split = income[income['age'] > age_median]

age_median_left_prob = age_left_split.shape[0] / income.shape[0]
age_median_right_prob = 1 - age_median_left_prob

In [46]:
age_information_gain = income_entropy -\
                        (age_median_left_prob*calc_entropy(age_left_split['high_income'])+\
                        age_median_right_prob*calc_entropy(age_right_split['high_income']))
age_information_gain

0.047028661304691965

## Finding the Best Split

Now that we know how to compute information gain, we can determine the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by calculating which split would have the highest information gain.

In [47]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set,
    column to split on, and target.
    """
    
    original_entropy = calc_entropy(data[target_name])
    
    column = data[split_name]
    median = column.median()
    
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # loop through the splits and calculate the subset entropies.
    to_substract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0]/data.shape[0])
        to_substract += prob * calc_entropy(subset[target_name])
    
    return original_entropy - to_substract

In [48]:
# verify.
print(calc_information_gain(income, 'age', 'high_income'))

0.0470286613047


Create a list called `information_gains`.
* It should contain, in order, the information gain from splitting on these columns: 
   * `age`, `workclass`, `education_num`, `marital_status`, `occupation`, `relationship`, `race`, `sex`, `hours_per_week`, `native_country`.

Find the highest value in the `information_gains` list, and assign the corresponding column name to `highest_gain`.

In [50]:
cols_to_split = ['age', 'workclass', 'education_num',
               'marital_status', 'occupation', 'relationship',
               'race', 'sex', 'hours_per_week', 'native_country']
information_gains = []

for col in cols_to_split:
    ig = calc_information_gain(income, col, 'high_income')
    information_gains.append(ig)
    
information_gains

[0.047028661304691965,
 0.0068109840543966182,
 0.065012984132774232,
 0.1114272573715438,
 0.0015822303843424645,
 0.047362416650269412,
 0.0,
 0.0,
 0.040622468671234868,
 0.00013457344495848567]

In [54]:
highest_gain = cols_to_split[information_gains.index(max(information_gains))]
highest_gain

'marital_status'

## Build the Whole Tree

Now we know how to make a single split. To construct an entire tree, we'll need to continue creating splits until the leaves only have a single class.<br>

Here's an example of what this looks like:

```python
age    high_income
25     1
50     1
30     0
50     0
80     1
```

![](img/5.png)

As you can see, we split the data all the way down the tree to the leaves. This example is different, because we only have a single variable to split with. This results in one leaf where both rows have an age of `50`, so we can't split. One has a `high_income` of `1`, however, and the other has a `high_income` of `0`. That means we'd usually be able to split on another variable, but that's not possible in this case. Instead, we predict `.5` for any rows that end up at this leaf.<br>

In order to construct a tree like this, we'll need to be able to split each node multiple times.





## Next Steps

So far, we've been following the [ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm) to construct decision trees. There are other algorithms like [CART](ftp://public.dhe.ibm.com/software/analytics/spss/support/Stats/Docs/Statistics/Algorithms/13.0/TREE-CART.pdf) that use different measures for the split criterion. We'll learn more about these other algorithms in future missions.<br>

In the next mission, we'll figure out how to recursively construct an entire tree using the ID3 algorithm.